In [57]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter

In [50]:
def process_data_for_labels(ticker):
    hm_days=7
    df = pd.read_csv('sp500_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    #print(tickers)
    df.fillna(0, inplace=True)
    for i in range(1, hm_days + 1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker] 
    df.fillna(0, inplace=True)
    return tickers, df

In [51]:
tickers, df = process_data_for_labels('AMZN')
print('df shape', df.shape)
print('df null shape', df.isnull().shape)
df.head(n=3)
df.count()
print('nan', np.count_nonzero(df.isnull()))

df shape (4527, 57)
df null shape (4527, 57)
nan 0


In [100]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.03
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0
    

In [98]:
def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)
    diff_1to7d= [df['{}_{}d'.format(ticker,i)] for i in range(1,8)]
    df['{}_target'.format(ticker)] = list(map(buy_sell_hold, * diff_1to7d))
    
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals =[str(i) for i in vals]
    print('Data spred:', Counter(str_vals))
    df.fillna(0, inplace = True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    
    print('df shape', df.shape)
    df_vals = df[[ticker for ticker in tickers]].pct_change()
    print('df_val shape', df_vals.shape)
    df_vals=df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)
    
    X = df_vals.values
    y = df['{}_target'.format(ticker)].values
    return X, y, df

In [99]:
X, y, df = extract_featuresets('AMZN')
print('X shape', X.shape)
print('y shape', y.shape)

Data spred: Counter({'1': 1944, '-1': 1671, '0': 912})
df shape (4527, 58)
df_val shape (4527, 50)
X shape (4527, 50)
y shape (4527,)


In [86]:
list(map(buy_sell_hold, [1,2]))

args [1]
args [2]


[1, 1]